# Cubeviz Demonstration Notebook

This notebook demonstrates the Cubeviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Cubeviz, can be found here: https://jdaviz.readthedocs.io/en/latest/cubeviz/

Import modules needed for this notebook.

In [ ]:
import warnings

from photutils.aperture import CircularAperture
from regions import PixCoord, CirclePixelRegion

from jdaviz import Cubeviz

We create a Cubeviz instance and show it.

In [ ]:
cubeviz = Cubeviz()
cubeviz.show()

Now we load observations. If you already have the files on your local machine, you can 
load them by passing their file paths to `load` as strings. For this example, 
we will retrieve remote data from [MAST](https://mast.stsci.edu/) via `astroquery`
by passing the observation's URI as a string. By default, the downloaded files are 
saved to the current working directory. If you set `cache=False` in the `load` 
call, it will re-download the file if desired.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. Again, you can turn off caching by setting `cache=False` to force it to re-download the 
file.

In [ ]:
filestem = "jw02732-c1001_t004_miri_ch1-short_s3d"
uri = f"mast:JWST/product/{filestem}.fits"

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load(uri, format='3D Spectrum', cache=True)

Alternatively, the data and the configuration can be autodetected and loaded simultaneously by calling `import jdaviz; jdaviz.open(uri or filename)`

The viewer containing the FLUX cube is the default viewer and is accessible as follows.

In [ ]:
viewer = cubeviz.default_viewer

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the circular selection tool (<img
  src="data:image/svg+xml;utf8,<svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 23.53 24.66' aria-hidden='true'><path d='M14.44,7.27a7,7,0,1,0,7,7A7,7,0,0,0,14.44,7.27Zm-1,11h0A1.48,1.48,0,0,1,12,16.76h0a1.48,1.48,0,0,1,1.48-1.48h0a1.48,1.48,0,0,1,1.48,1.48h0A1.48,1.48,0,0,1,13.44,18.24Zm2.11-4.5h0a1.48,1.48,0,0,1-1.48-1.48h0a1.48,1.48,0,0,1,1.48-1.48h0A1.48,1.48,0,0,1,17,12.26h0A1.48,1.48,0,0,1,15.55,13.74Z' fill='%23000' stroke='%23fff' stroke-width='1.5' stroke-linejoin='round' stroke-linecap='round' vector-effect='non-scaling-stroke' style='paint-order:stroke fill'/><rect x='2.48' y='16.18' width='2.96' height='2.96' rx='1.48' fill='%23000' stroke='%23fff' stroke-width='1.5' stroke-linejoin='round' vector-effect='non-scaling-stroke' style='paint-order:stroke fill'/><rect x='4.48' y='3.3' width='2.96' height='2.96' rx='1.48' fill='%23000' stroke='%23fff' stroke-width='1.5' stroke-linejoin='round' vector-effect='non-scaling-stroke' style='paint-order:stroke fill'/><rect x='9.47' y='1.78' width='2.96' height='2.96' rx='1.48' fill='%23000' stroke='%23fff' stroke-width='2' stroke-linejoin='round' vector-effect='non-scaling-stroke' style='paint-order:stroke fill'/></svg>"
  width="16"
  style="vertical-align:-4px;"
/>), and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = cubeviz.plugins['Subset Tools'].get_regions(region_type="spatial")

In [ ]:
regions

It is also possible to programmatically pass a `regions` shape or a `photutils` aperture shape.

*NOTE: Sky regions are not yet supported for data cubes.*

In [ ]:
data = cubeviz.get_data(filestem)

In [ ]:
# photutils aperture
my_aper = CircularAperture((15, 10), r=5)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=35, y=35), radius=10)

my_regions = [my_aper, my_reg]
cubeviz.plugins['Subset Tools'].import_region(my_regions, combination_mode='new')

To get the spectra:

In [ ]:
spectra_dict = cubeviz.specviz.get_spectra(apply_slider_redshift=True)
spectra_dict

This contains the masked spectrum extracted from "Subset 1" region.

In [ ]:
subset1_spec1d = spectra_dict['Spectrum (Subset 1, sum)']
subset1_spec1d